## Creation of  a Counting Model with fixed background

In this notebook we create a Poisson counting model with fixed background.



The model describing the experiment is described by a Poisson distribution:  

$$ P(n_{obs} | s,b) = Poisson (n_{obs} | s+b) = (s+b)^{n_{obs} } \frac{ e^{-(s+b)} } { n_{obs} ! } $$

We consider that we know precisly the number of background events. We will see in the next example how we can include in the model an uncertainty in $b$. 


We will create the model using **RooFit** and **RooStats**. 
See the slides 
-  RooFit: https://cernbox.cern.ch/index.php/s/kzGv4406IrpFpVM
-  RooStats: https://cernbox.cern.ch/index.php/s/9DWJCObingTWB3r

In [ ]:
int nobs; double b; 

In [ ]:
nobs = 4;                // number of observed events
b = 3;                 // number of expected background even

#### Model Creation using `RooWorkspace` 

Using the factory syntax of the `RooWorkspace` we create the parameters of the model and the Poisson p.d.f describing the model.

In [ ]:
RooWorkspace w("w");
   
// make Poisson model * Gaussian constraint
w.factory("sum:nexp(s[3,0,15],b[1])");
// Poisson of (n | s+b)
w.factory("Poisson:model(nobs[0,50],nexp)");

#### Creation of  the  `RooStats::ModelConfig` class

We create the `ModelConfig` class, which is needed to run all `RooStats` statistics calculators. 
The class contains meta-information on the model, such as the name of the pdf, the observables, the parameter of interest and if existing the nuisance parameters. 

In [ ]:
RooStats::ModelConfig mc("ModelConfig",&w);
mc.SetPdf(*w.pdf("model"));
mc.SetParametersOfInterest(*w.var("s"));
mc.SetObservables(*w.var("nobs"));

// these are needed for the hypothesis tests
mc.SetSnapshot(*w.var("s"));

mc.Print();
// import model in the workspace 
w.import(mc);

#####  Setting  the model parameter values  ( e.g. number of background events )

We set the assumed value of b, and  since we consider it fixed (known precisly) it is important to set it constant.  

In [ ]:
w.var("b")->setConstant(true); 
w.var("b")->setVal(b);    

#### Data Set generation

We create here the observed data set for the model. The data set will conist of a single event with the observable 
$n_{obs}$ equal to the given value (e.g. 3). 

It is important to import the data set in the workspace


In [ ]:
RooDataSet data("data","", *w.var("nobs"));
w.var("nobs")->setVal(nobs);
data.add(*w.var("nobs") );
// import data set in workspace 
w.import(data);
std::cout << "number of observed events  = " << data.get(0)->getRealValue("nobs") << std::endl;

#### Save Workspace in a file

We save the generated model as a workspace in a file that can be used later on. 
Before saving we print its content for showing its content. 

In [ ]:
w.Print();
w.writeToFile("CountingModelFixedB.root", true);